In [90]:
from PIL import Image
def convert_file_to_binary(path):
    with open(path, 'rb') as f:
        binary_data = f.read()
        binary_string = ''.join(format(byte, '08b') for byte in binary_data)
    print(f'Binary length : {8*len(binary_string)}')
    return binary_string


In [97]:
def create_image(name_of_file,img_index,current_index,binary_string):
    new_image = Image.new('RGB', (7680, 4320), (0, 0, 0))

    # Get the size of the image
    width, height = new_image.size

    # Loop over each pixel in the image and set its value
    # current_index=int(current_index)
    for x in range(width):
        for y in range(height):
            # Set the color of the pixel
            if(current_index<len(binary_string)):
                if(binary_string[current_index]=='1'):
                    new_image.putpixel((x, y), (256, 256, 256))  # white
                elif(binary_string[current_index]=='0'):
                    new_image.putpixel((x, y), (0, 0, 0))  # black
                current_index+=1
            else:
                new_image.putpixel((x, y), (0, 0, 256))

            

    # Save the new image
    new_image.save('./images/'+name_of_file+'_pixels_'+'{:04d}'.format(img_index)+'.jpg')
    return current_index
    

In [103]:
def Create_images(name_of_file,binary_string):
    total_length=len(binary_string)
    img_index=1
    current_index=0
    while(total_length>33177600):
        current_index=create_image(name_of_file,img_index,current_index,binary_string)
        img_index+=1
        
    if(current_index<33177600):
        create_image(name_of_file,img_index,current_index,binary_string)

In [104]:
def cal_no_of_images(name_of_file):
    import os
    directory = './images/'
    file_list = os.listdir(directory)
    num_files=0
    for file in file_list:
        if name_of_file == str(file)[:-16]:
            print("images are : "+str(file))
            num_files+=1

    return num_files


In [105]:
def CreateVideo(name_of_file):
    import imageio.v2
    writer = imageio.v2.get_writer('./videos/'+name_of_file+'.mp4', fps=1)
    noi=cal_no_of_images(name_of_file)
    for i in range(noi):
        image = imageio.v2.imread('./images/'+name_of_file+'_pixels_'+'{:04d}'.format(i+1)+'.jpg')
        writer.append_data(image)

    writer.close()


In [106]:
def convertdatatovideo(name_of_file,path):
    binary_string=convert_file_to_binary(path)
    Create_images(name_of_file,binary_string)
    CreateVideo(name_of_file)
    

In [107]:
# dot_index = s.rfind('.')
import os
directory = './inputs/'
file_list = os.listdir(directory)
for file in file_list:
    convertdatatovideo(str(file),directory+str(file))

Binary length : 138898816
images are : dependency-check.html_pixels_0001.jpg


[rawvideo @ 0x73d8680] Stream #0: not enough frames to estimate rate; consider increasing probesize


Binary length : 137792
images are : myfile.txt_pixels_0001.jpg


[rawvideo @ 0x723a680] Stream #0: not enough frames to estimate rate; consider increasing probesize
